# Post-wildfire Vegetation Recovery
## In this Jupyter notebook, we explore the NEON Terrestrial Observation System plots. 
* First, we determine which NEON plots are within the Chimney Tops 2 Fire boundary. 
* Next, we determine which NEON plots were classified for vegetation and percent cover and find their spatial coordinates using the NEON API. 
* Lastly, we analyze subplots of plot 55 for percent cover and plot our results using a pivot table.

Author: Heidi Yoon

In [ ]:
# Import packages
import os
import sys

import matplotlib.pyplot as plt
import pandas as pd
import requests
import geopandas as gpd
import earthpy as et

# Check module path
project_path = os.path.abspath(os.path.join('..'))
if project_path not in sys.path:
    sys.path.append(project_path)
    
import modules.reflectance as refl
import modules.plots as plots

In [ ]:
# Get NEON plant presence percent cover data from NEON API
# Endpoints are data product, site, date, and release year
neon_base = "https://data.neonscience.org/api/v0/data"
data_product = "/DP1.10058.001"
site = "/GRSM"
april_release = "/2017-04?package=expanded&release=RELEASE-2022"
july_release = "/2017-07?package=expanded&release=RELEASE-2022" 
aug_release = "/2017-08?package=expanded&release=RELEASE-2022"
april_url = neon_base + data_product + site + april_release
july_url = neon_base + data_product + site + july_release
aug_url = neon_base + data_product + site + aug_release
april_data = requests.get(april_url)
july_data = requests.get(july_url)
aug_data = requests.get(aug_url)

In [ ]:
# Using API response, find the url corresponding to 1m2 subplots
april_df = pd.json_normalize(april_data.json()['data']['files'])
july_df = pd.json_normalize(july_data.json()['data']['files'])
aug_df = pd.json_normalize(aug_data.json()['data']['files'])

In [ ]:
# Define data directory and data paths
data_dir = os.path.join(project_path, 'data')
plots_dir = os.path.join(data_dir, 'neon_tos_plot_centroids')

april_plots_path = os.path.join(data_dir, 'NEON_GRSM_201704_1m2_veg_cover.csv') 
july_plots_path = os.path.join(data_dir, 'NEON_GRSM_201707_1m2_veg_cover.csv')
aug_plots_path = os.path.join(data_dir, 'NEON_GRSM_201708_1m2_veg_cover.csv')

chimney_fire_dir = os.path.join(data_dir, 'chimtops2_burn_bndy')
chimney_fire_path = os.path.join(
    data_dir, 'chimtops2_burn_bndy', 'chimtops2_burn_bndy.shp')
plots_path = os.path.join(
    data_dir, 'neon_tos_plot_centroids', 'ALL_NEON_TOS_Plot_Centroids_V8.shp')

# Check if data directory exists
try:
    os.makedirs(data_dir)
    print('The data directory has been created!')
except FileExistsError:
    print('The data directory already exists!')

In [ ]:
# Get NEON plant presence percent cover data
april_csv = (
    'NEON.D07.GRSM.DP1.10058.001.div_1m2Data.2017-04.expanded.20211221T195314Z.csv')
july_csv = (
    'NEON.D07.GRSM.DP1.10058.001.div_1m2Data.2017-07.expanded.20211222T004157Z.csv')
aug_csv = (
    'NEON.D07.GRSM.DP1.10058.001.div_1m2Data.2017-08.expanded.20211222T012732Z.csv')

april_plots_url = april_df[april_df['name'] == april_csv]['url'].iloc[0]
july_plots_url = july_df[july_df['name'] == july_csv]['url'].iloc[0]
aug_plots_url = aug_df[aug_df['name'] == aug_csv]['url'].iloc[0]

april_data = refl.download_file(april_plots_path, april_plots_url)
july_data = refl.download_file(july_plots_path, july_plots_url)
aug_data = refl.download_file(aug_plots_path, aug_plots_url)

In [ ]:
# Get MTBS fire boundary
fire_bound_url = (
        'https://github.com/AreteY/post-wildfire-vegetation-change/files/'
        '8567790/chimtops2_burn_bndy.zip')

fire_bound_data = refl.download_zipfile(
    chimney_fire_path, fire_bound_url, chimney_fire_dir)

# Get NEON TOS plot centroids
plots_url = (
    'https://github.com/AreteY/post-wildfire-recovery/files/8961276/'
    'neon_tos_plot_centroids.zip')
plots_data = refl.download_zipfile(plots_path, plots_url, plots_dir)

In [ ]:
# Read data
fire_bound = gpd.read_file(chimney_fire_path)
neon_plots = gpd.read_file(plots_path)

# Reproject CRS to UTM 
fire_reproj = fire_bound.to_crs(epsg=32617)
plots_reproj = neon_plots.to_crs(epsg=32617)

### Since the shapefile contains all of the NEON TOS plots, we crop the plots shapefile to the fire boundary and determine which plots are inside the fire boundary by using the `geopandas.sjoin()` function.

In [ ]:
# Clip plots to fire boundary
plots_clip = gpd.clip(plots_reproj, fire_reproj)

# Assign plots a fire boundary attribute
plots_fire_bound = gpd.sjoin(plots_clip,
                             fire_reproj,
                             how='inner',
                             predicate='intersects')

### We can list the plots that are located inside the fire boundary:

In [ ]:
#sorted(plots_fire_bound['plotID'].unique())

### We can also visualize the plots located inside the fire boundary in a map:

In [ ]:
# Plot the fire perimeter within NEON plots
fig, ax = plt.subplots(figsize=(12, 12))

fire_reproj.plot(ax=ax,
                 color='white',
                 edgecolor='black')

plots_fire_bound.plot(ax=ax,
                      color='yellow',
                      edgecolor='purple')


# Set figure title and axes 
ax.set(
   title="Map",
   xlabel='UTM easting (m)',
   ylabel='UTM northing (m)')
ax.ticklabel_format(useOffset=False, style='plain')

plt.show()

### Not all of the plots within the fire boundary are classified for plant presence and percent cover by the NEON Terrestrial Observation System team. We can determine which plots are classified by using the `pandas.merge()` function with the plots within the fire boundary and the classified plots from the plant presence and percent cover data product.

In [ ]:
# Read NEON plant presence percent cover data
veg_201704_1m2_df = pd.read_csv(april_plots_path)
veg_201707_1m2_df = pd.read_csv(july_plots_path)
veg_201708_1m2_df = pd.read_csv(aug_plots_path)

In [ ]:
april_fire_plots = pd.merge(
    veg_201704_1m2_df, plots_fire_bound, how='inner',
    left_on='plotID', right_on='plotID'
)
april_fire_plots['plotID'].unique()

In [ ]:
july_fire_plots = pd.merge(
    veg_201707_1m2_df, plots_fire_bound, how='inner',
    left_on='plotID', right_on='plotID'
)
july_fire_plots['plotID'].unique()

In [ ]:
aug_fire_plots = pd.merge(
    veg_201708_1m2_df, plots_fire_bound, how='inner',
    left_on='plotID', right_on='plotID'
)
aug_fire_plots['plotID']

### We find that 9 classified plots are within the fire boundary. We can find the coordinates of each of the subplots within these plots using the NEON API.

In [ ]:
# Initialize plots list
plots_ls = []

# Define plots and subplots within fire perimeter
fire_plots = ['/GRSM_001.basePlot.div', '/GRSM_003.basePlot.div',
              '/GRSM_007.basePlot.div', '/GRSM_008.basePlot.div',
              '/GRSM_009.basePlot.div', '/GRSM_012.basePlot.div',
              '/GRSM_055.basePlot.div', '/GRSM_058.basePlot.div',
              '/GRSM_059.basePlot.div']

sub_plots = ['.40.1.1', '.40.3.1', '.41.1.1', '.41.4.1', 
             '.31.1.1', '.31.4.1', '.32.2.1', '.32.4.1']

# NEON API for UTM coordinates of TOS plots in GRSM
# Endpoint is location and location name = fire plots
neon_base = "https://data.neonscience.org/api/v0"
data_product = "/locations"

In [ ]:
for plot in fire_plots:
    for sub in sub_plots:
        plot_url = neon_base + data_product + plot + sub
        plot_data = requests.get(plot_url)
        grsm_plots_df = pd.json_normalize(plot_data.json()['data'])
        plots_ls += [[grsm_plots_df['locationName'][0], 
                      grsm_plots_df['locationUtmEasting'][0], 
                      grsm_plots_df['locationUtmNorthing'][0]]]
plots_df = pd.DataFrame(
    plots_ls, columns = ['name', 'utm-easting', 'utm-northing'])
grsm_plots_path = os.path.join(
    project_path, 'outputs', 'grsm_plots_coords.csv')
plots_df
#plots_df.to_csv(grsm_plots_path, index=False)

### For each plot, we can analyze each of the subplots and determine which percentage contains vegetation, standing dead trees, soil, and litter. For this example, let's consider plot 55 and use the field data from April 2017.

In [ ]:
grsm_055_201704_all = veg_201704_1m2_df[veg_201704_1m2_df['namedLocation']
                                        == 'GRSM_055.basePlot.div']
grsm_055_201704 = grsm_055_201704_all[[
    'plotID', 'subplotID', 'family', 'otherVariables', 'percentCover']]

### The vegetation in each subplot is identified by the species family. Other components are classified into categories, such as, standing dead, soil, litter, lichen, moss, rock, wood, and other non-vascular cover. 
Here is an example of the percent cover for plot 55 subplot 31.1.1:

In [ ]:
# Dataframe for subplot 31.1.1 - vegetation and other components
grsm_055_3111_201704 = grsm_055_201704[grsm_055_201704['subplotID']=='31.1.1']
# Subset of dataframe for other components
grsm_055_3111_othervar = grsm_055_3111_201704.dropna(subset=['otherVariables'])
grsm_055_3111_201704

### Using dataframes subsetted for each subplot, we can calculate the percent cover. Using the function `percent_cover_by_plot` below, we report the vegetation percent cover as the sum of the percent cover of all identified vegetation. The categories for standing dead trees, soil, and litter are weighted by the non-vegetation fraction.

In [ ]:
# Percent Cover for all subplots within Plot 55 for April 2017
plot055 = 'GRSM_055.basePlot.div'
plots.percent_cover_by_plot(april_plots_path, plot055)

### We reorder the data frame to create a pivot table below for the left-most subplots (40.1.1, 40.3.1, and 31.1.1), the right-most subplots (41.4.1, 32.4.1, 32.2.1), and the middle subplots (31.4.1, 41.1.1).

In [ ]:
a = ['veg', '40.1.1', 13.5]
b = ['st.dead', '40.1.1', 0.9]
c = ['soil', '40.1.1', 25.1]
d = ['litter', '40.1.1', 34.6]
e = ['veg', '40.3.1', 23.0]
f = ['st.dead', '40.3.1', 1.5]
g = ['soil', '40.3.1', 40.8]
h = ['litter', '40.3.1', 23.1]
i = ['veg', '31.1.1', 20.5]
j = ['st.dead', '31.1.1', 0.8]
k = ['soil', '31.1.1', 59.6]
l = ['litter', '31.1.1', 15.9]
left_plots = [a, b, c, d, e, f, g, h, i, j, k, l]
left_plots_df = pd.DataFrame(
    left_plots, columns=['class', 'subplot', 'percent'])
left_plots_df = left_plots_df.set_index('class')
left_plots_pivot = left_plots_df.pivot_table('percent', ['class'], 'subplot')
left_plots_pivot

In [ ]:
aa = ['veg', '41.4.1', 34.5]
bb = ['st.dead', '41.4.1', 0.7]
cc = ['soil', '41.4.1', 45.2]
dd = ['litter', '41.4.1', 15.1]
ee = ['veg', '32.4.1', 21.0]
ff = ['st.dead', '32.4.1', 0.8]
gg = ['soil', '32.4.1', 15.8]
hh = ['litter', '32.4.1', 46.6]
ii = ['veg', '32.2.1', 22.0]
jj = ['st.dead', '32.2.1', 0.8]
kk = ['soil', '32.2.1', 26.5]
ll = ['litter', '32.2.1', 35.1]
right_plots = [aa, bb, cc, dd, ee, ff, gg, hh, ii, jj, kk, ll]
right_plots_df = pd.DataFrame(
    right_plots, columns=['class', 'subplot', 'percent'])
right_plots_df = right_plots_df.set_index('class')
right_plots_pivot = right_plots_df.pivot_table('percent', ['class'], 'subplot')
right_plots_pivot

In [ ]:
aaa = ['veg', '31.4.1', 27.5]
bbb = ['st.dead', '31.4.1', 0.0]
ccc = ['soil', '31.4.1', 58.0]
ddd = ['litter', '31.4.1', 5.8]
eee = ['veg', '41.1.1', 23.5]
fff = ['st.dead', '41.1.1', 0.0]
ggg = ['soil', '41.1.1', 42.8]
hhh = ['litter', '41.1.1', 25.2]
middle_plots = [aaa, bbb, ccc, ddd, eee, fff, ggg, hhh]
middle_plots_df = pd.DataFrame(
    middle_plots, columns=['class', 'subplot', 'percent'])
middle_plots_df = middle_plots_df.set_index('class')
middle_plots_pivot = middle_plots_df.pivot_table('percent', ['class'], 'subplot')
middle_plots_pivot

In [ ]:
# Plot percent cover for plot 55
left_colors = ["green", "blue", "purple"]
right_colors = ["green", "purple", "blue"]
middle_colors = ["blue", "purple"]

# Define plot space
f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 12))

left_plots_pivot.plot.bar(color=left_colors,
                          ax=ax1).legend(loc='upper right')
right_plots_pivot.plot.bar(color=right_colors,
                           ax=ax2).legend(loc='upper right')
middle_plots_pivot.plot.bar(color=middle_colors,
                            ax=ax3).legend(loc='upper right')

# Set figure title and axes labels
ax1.set(title="Percent Cover for Plot 55",
        ylabel="Percent Cover")
ax2.set(ylabel="Percent Cover")
ax3.set(xlabel="Classification",
        ylabel="Percent Cover")

ax1.set(ylim=[0, 65])
ax2.set(ylim=[0, 65])
ax3.set(ylim=[0, 65])

plt.setp(ax1.get_xticklabels(), rotation = 45)
plt.setp(ax2.get_xticklabels(), rotation = 45)
plt.setp(ax3.get_xticklabels(), rotation = 45)

plt.tight_layout

plt.show()